In [23]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
__author__ = "Jennifer Reisinger"
__email__ = "jhubbard3@students.columbiabasin.edu"
__date__ = "Spring 2023"
__version__ = "0.0.1"

#libraries
import pandas as pd
import hashlib

In [24]:
#read the csv file into pandas
df = pd.read_csv('COVID-19_Hospital_Impact_data.csv', usecols=['hospital_pk','collection_week','state','ccn',
        'hospital_name','address','city','zip','hospital_subtype','fips_code','is_metro_micro'])
df.head(10)

,hospital_pk,collection_week,state,ccn,hospital_name,address,city,zip,hospital_subtype,fips_code,is_metro_micro
0,161324,3/4/22,IA,161324,GEORGE C GRAPE COMMUNITY HOSPITAL,2959 US HIGHWAY 275,HAMBURG,51640.0,Critical Access Hospitals,19071.0,False
1,431312,3/4/22,SD,431312,MARSHALL COUNTY HEALTHCARE CENTER - CAH,413 9TH STREET,BRITTON,57430.0,Critical Access Hospitals,46091.0,False
2,51321,2/25/22,CA,51321,HEALDSBURG HOSPITAL,1375 UNIVERSITY AVENUE,HEALDSBURG,95448.0,Critical Access Hospitals,6097.0,True
3,100048,2/25/22,FL,100048,JAY HOSPITAL,14114 ALABAMA ST,JAY,32565.0,Short Term,12113.0,True
4,10T290,2/25/22,FL,10T290,TVRH REHABILITATION HOSPITAL,1451 EL CAMINO REAL,THE VILLAGES,32159.0,Short Term,12119.0,True
5,241308,2/25/22,MN,241308,LAKE VIEW MEMORIAL HOSPITAL,325 ELEVENTH AVE,TWO HARBORS,55616.0,Critical Access Hospitals,27075.0,True
6,30137,9/4/20,AZ,30137,SANTA CRUZ VALLEY REGIONAL HOSPITAL,4455 SOUTH I-19 FRONTAGE ROAD,GREEN VALLEY,85614.0,Short Term,4019.0,True
7,271308,2/25/22,MT,271308,ROOSEVELT MEDICAL CENTER,818 2ND AVE E,CULBERTSON,59218.0,Critical Access Hospitals,30085.0,False
8,380071,9/4/20,OR,380071,WILLAMETTE VALLEY MEDICAL CENTER,2700 SE STRATUS AVE.,MCMINNVILLE,97128.0,Short Term,41071.0,True
9,280127,2/25/22,NE,280127,LINCOLN SURGICAL HOSPITAL,1710 SOUTH 70TH STREET,LINCOLN,68506.0,Short Term,31109.0,True


In [25]:
#R1 - Filter the results to be able to search by State AND a partial zip code
state_filter = df['state'] == 'OR'
zip_filter = df['zip'].astype(str).str[:3]=='972'
df.loc[state_filter & zip_filter]

,hospital_pk,collection_week,state,ccn,hospital_name,address,city,zip,hospital_subtype,fips_code,is_metro_micro
73,382004,2/11/22,OR,382004,VIBRA SPECIALTY HOSPITAL OF PORTLAND,10300 NE HANCOCK STREET,PORTLAND,97220.0,Long Term,41051.0,True
1721,383300,3/4/22,OR,383300,SHRINERS HOSPITAL FOR CHILDREN-PORTLAND,3101 SW SAM JACKSON PARK ROAD,PORTLAND,97239.0,Childrens Hospitals,41051.0,True
2406,380082,3/4/22,OR,380082,PROVIDENCE MILWAUKIE HOSPITAL,10150 SE 32ND AVENUE,MILWAUKIE,97222.0,Short Term,41005.0,True
2440,382004,3/4/22,OR,382004,VIBRA SPECIALTY HOSPITAL OF PORTLAND,10300 NE HANCOCK STREET,PORTLAND,97220.0,Long Term,41051.0,True
2453,380009,3/4/22,OR,380009,OHSU HOSPITAL AND CLINICS,3181 SW SAM JACKSON PARK ROAD,PORTLAND,97239.0,Short Term,41051.0,True
...,...,...,...,...,...,...,...,...,...,...,...
418671,380004,8/7/20,OR,380004,PROVIDENCE ST VINCENT MEDICAL CENTER,9205 SW BARNES ROAD,PORTLAND,97225.0,Short Term,41067.0,True
418927,380004,7/31/20,OR,380004,PROVIDENCE ST VINCENT MEDICAL CENTER,9205 SW BARNES ROAD,PORTLAND,97225.0,Short Term,41067.0,True
419148,380061,8/7/20,OR,380061,PROVIDENCE PORTLAND MEDICAL CENTER,4805 NE GLISAN STREET,PORTLAND,97213.0,Short Term,41051.0,True
419412,383300,7/31/20,OR,383300,SHRINERS HOSPITAL FOR CHILDREN-PORTLAND,3101 SW SAM JACKSON PARK ROAD,PORTLAND,97239.0,Childrens Hospitals,41051.0,True


In [35]:
#R2 Create a “signature” for each hospital.

# def hash_pair() converts the pair into strings before creating hashed objects and returning the hash object.
def hash_pair(pair):
    pair_string=str(pair[0]) + str(pair[1])
    hash_object = hashlib.sha256(pair_string.encode())
    return hash_object.hexdigest()

#generating a dictionary where the pairs would act as keys and indices as values & creates a unique identifier 
unique_pairs = set(tuple(x) for x in df[['hospital_pk', 'collection_week']].values)
hash_dict = {pair: hash_pair(pair) for pair in unique_pairs}
df['hospital_hash'] = [hash_dict[tuple(x)] for x in df[['hospital_pk', 'collection_week']].values]

In [36]:
#Check for duplicate signatures in 'hospital_hash' by grouping & filtering.
duplicates = df.groupby('hospital_hash').filter(lambda x: len(x) > 1)
print(duplicates)


Empty DataFrame
Columns: [hospital_pk, collection_week, state, ccn, hospital_name, address, city, zip, hospital_subtype, fips_code, is_metro_micro, hospital_hash]
Index: []


In [37]:
# Use filtering to be able to search for a particular hospital pk, or a portion of a hospital hash as performed with R1 state/zip
# changing the hospital_hash value to a string datatype, and searching the first 4 places of the hospital hash for "69d3*" OR
# for hospital_pk = 380004. Results will show for both searches, but can be easily changed by swapping out the hospital_pk== 'y' with a 
# different y, OR changing the str[:x]=='n' to the appropriate 'x' and 'n' values.

hospitalpk_filter = df['hospital_pk'] == '380004'
hospitalhash_filter = df['hospital_hash'].astype(str).str[:4]=='69d3'
df.loc[hospitalpk_filter | hospitalhash_filter]


,hospital_pk,collection_week,state,ccn,hospital_name,address,city,zip,hospital_subtype,fips_code,is_metro_micro,hospital_hash
7183,380004,2/25/22,OR,380004,PROVIDENCE ST VINCENT MEDICAL CENTER,9205 SW BARNES ROAD,PORTLAND,97225.0,Short Term,41067.0,True,4a9b9ecd1fb19ed81fa279c427a05ea3b71ba48486c0eb...
12815,380004,3/4/22,OR,380004,PROVIDENCE ST VINCENT MEDICAL CENTER,9205 SW BARNES ROAD,PORTLAND,97225.0,Short Term,41067.0,True,6b5d03072a7837172f17486412fad80ef025c234f4ac48...
14868,380004,2/4/22,OR,380004,PROVIDENCE ST VINCENT MEDICAL CENTER,9205 SW BARNES ROAD,PORTLAND,97225.0,Short Term,41067.0,True,b4040f1fddc6d380dcdcdedd4bd677d8875047d7e3ca62...
20211,380004,2/11/22,OR,380004,PROVIDENCE ST VINCENT MEDICAL CENTER,9205 SW BARNES ROAD,PORTLAND,97225.0,Short Term,41067.0,True,492810a01954cccca5cc02a5e844fae836ed67508d426b...
22116,380004,2/18/22,OR,380004,PROVIDENCE ST VINCENT MEDICAL CENTER,9205 SW BARNES ROAD,PORTLAND,97225.0,Short Term,41067.0,True,69b5ec86c8cb899ba6e04daf7aad3d89c913cf37923126...
...,...,...,...,...,...,...,...,...,...,...,...,...
396046,380004,9/11/20,OR,380004,PROVIDENCE ST VINCENT MEDICAL CENTER,9205 SW BARNES ROAD,PORTLAND,97225.0,Short Term,41067.0,True,cb6043659430a094edfa8d9bb04147da084737b188c34b...
396392,380004,8/28/20,OR,380004,PROVIDENCE ST VINCENT MEDICAL CENTER,9205 SW BARNES ROAD,PORTLAND,97225.0,Short Term,41067.0,True,834fb4ad880625df17d59dca37bce1783df12af52d6a24...
407403,380004,8/21/20,OR,380004,PROVIDENCE ST VINCENT MEDICAL CENTER,9205 SW BARNES ROAD,PORTLAND,97225.0,Short Term,41067.0,True,1cb011901a8309d633086a7d05fbf29ff90e9b1dbeb994...
418671,380004,8/7/20,OR,380004,PROVIDENCE ST VINCENT MEDICAL CENTER,9205 SW BARNES ROAD,PORTLAND,97225.0,Short Term,41067.0,True,741645e60fb9d9a467ee1e9b03d3fb6acf8adeb9280998...


In [38]:
# R3 Devise an algorithm to show "similar" hospitals to a selected hospital_pk. 

# Load more columns into the dataframe
df2 = pd.read_csv('COVID-19_Hospital_Impact_data.csv', usecols=['hospital_pk','collection_week','state','ccn',
        'hospital_name','address','city','zip','hospital_subtype','fips_code','is_metro_micro','all_adult_hospital_beds_7_day_avg','all_adult_hospital_inpatient_beds_7_day_avg',
        'inpatient_beds_used_7_day_avg','inpatient_beds_used_covid_7_day_avg','inpatient_beds_7_day_avg','total_icu_beds_7_day_avg', 'total_staffed_adult_icu_beds_7_day_avg','icu_beds_used_7_day_avg',
        'icu_patients_confirmed_influenza_7_day_avg','total_beds_7_day_sum','all_adult_hospital_beds_7_day_sum','inpatient_beds_used_7_day_sum',
        'inpatient_beds_7_day_sum','total_icu_beds_7_day_sum','icu_beds_used_7_day_sum','total_beds_7_day_coverage','staffed_adult_icu_bed_occupancy_7_day_avg'])
df2.head(20)

,hospital_pk,collection_week,state,ccn,hospital_name,address,city,zip,hospital_subtype,fips_code,...,icu_beds_used_7_day_avg,staffed_adult_icu_bed_occupancy_7_day_avg,icu_patients_confirmed_influenza_7_day_avg,total_beds_7_day_sum,all_adult_hospital_beds_7_day_sum,inpatient_beds_used_7_day_sum,inpatient_beds_7_day_sum,total_icu_beds_7_day_sum,icu_beds_used_7_day_sum,total_beds_7_day_coverage
0,161324,3/4/22,IA,161324,GEORGE C GRAPE COMMUNITY HOSPITAL,2959 US HIGHWAY 275,HAMBURG,51640.0,Critical Access Hospitals,19071.0,...,0.0,0.0,0.0,NaN,NaN,7.0,175.0,0.0,0.0,0
1,431312,3/4/22,SD,431312,MARSHALL COUNTY HEALTHCARE CENTER - CAH,413 9TH STREET,BRITTON,57430.0,Critical Access Hospitals,46091.0,...,0.0,0.0,0.0,NaN,NaN,18.0,50.0,0.0,0.0,0
2,51321,2/25/22,CA,51321,HEALDSBURG HOSPITAL,1375 UNIVERSITY AVENUE,HEALDSBURG,95448.0,Critical Access Hospitals,6097.0,...,0.0,0.0,0.0,NaN,NaN,76.0,119.0,35.0,0.0,0
3,100048,2/25/22,FL,100048,JAY HOSPITAL,14114 ALABAMA ST,JAY,32565.0,Short Term,12113.0,...,0.0,0.0,0.0,NaN,NaN,48.0,147.0,0.0,0.0,0
4,10T290,2/25/22,FL,10T290,TVRH REHABILITATION HOSPITAL,1451 EL CAMINO REAL,THE VILLAGES,32159.0,Short Term,12119.0,...,0.0,0.0,0.0,NaN,NaN,23.0,30.0,0.0,0.0,0
5,241308,2/25/22,MN,241308,LAKE VIEW MEMORIAL HOSPITAL,325 ELEVENTH AVE,TWO HARBORS,55616.0,Critical Access Hospitals,27075.0,...,0.0,0.0,0.0,NaN,NaN,68.0,85.0,0.0,0.0,0
6,30137,9/4/20,AZ,30137,SANTA CRUZ VALLEY REGIONAL HOSPITAL,4455 SOUTH I-19 FRONTAGE ROAD,GREEN VALLEY,85614.0,Short Term,4019.0,...,0.0,0.0,NaN,79.0,30.0,18.0,24.0,12.0,0.0,4
7,271308,2/25/22,MT,271308,ROOSEVELT MEDICAL CENTER,818 2ND AVE E,CULBERTSON,59218.0,Critical Access Hospitals,30085.0,...,0.0,0.0,0.0,NaN,NaN,133.0,175.0,0.0,0.0,0
8,380071,9/4/20,OR,380071,WILLAMETTE VALLEY MEDICAL CENTER,2700 SE STRATUS AVE.,MCMINNVILLE,97128.0,Short Term,41071.0,...,-999999.0,-999999.0,NaN,360.0,NaN,228.0,360.0,60.0,17.0,6
9,280127,2/25/22,NE,280127,LINCOLN SURGICAL HOSPITAL,1710 SOUTH 70TH STREET,LINCOLN,68506.0,Short Term,31109.0,...,0.0,0.0,0.0,NaN,NaN,9.0,125.0,0.0,0.0,0


In [15]:
# Check the datatypes of the columns
df2.dtypes

# Change the datatypes for the columns to strings, and filter as seen in #R1 & #R2 - Checking each column for the 1st number.
total_beds_7_day_coverage_filter = df2['total_beds_7_day_coverage'].astype(str).str[:1]=='4'
icu_beds_used_7_day_sum_filter = df2['icu_beds_used_7_day_sum'].astype(str).str[:1]=='0'
total_icu_beds_7_day_sum_filter = df2['total_icu_beds_7_day_sum'].astype(str).str[:1]=='0'
inpatient_beds_7_day_sum_filter = df2['inpatient_beds_7_day_sum'].astype(str).str[:1]=='0'
inpatient_beds_used_7_day_sum_filter = df2['inpatient_beds_used_7_day_sum'].astype(str).str[:1]=='0'
all_adult_hospital_beds_7_day_sum_filter = df2['all_adult_hospital_beds_7_day_sum'].astype(str).str[:1]=='0'
total_beds_7_day_sum_filter = df2['total_beds_7_day_sum'].astype(str).str[:1]=='0'
icu_patients_confirmed_influenza_7_day_avg_filter = df2['icu_patients_confirmed_influenza_7_day_avg'].astype(str).str[:1]=='0'
icu_beds_used_7_day_avg_filter = df2['icu_beds_used_7_day_avg'].astype(str).str[:1]=='0'
total_icu_beds_7_day_avg_filter = df2['total_icu_beds_7_day_avg'].astype(str).str[:1]=='0'
inpatient_beds_7_day_avg_filter = df2['inpatient_beds_7_day_avg'].astype(str).str[:1]=='0'

#display results
df2_filter = df2.loc[total_beds_7_day_coverage_filter & icu_beds_used_7_day_sum_filter & total_icu_beds_7_day_sum_filter & inpatient_beds_7_day_sum_filter
      & inpatient_beds_7_day_sum_filter & inpatient_beds_used_7_day_sum_filter & total_beds_7_day_sum_filter 
      & icu_patients_confirmed_influenza_7_day_avg_filter & icu_beds_used_7_day_avg_filter & total_icu_beds_7_day_avg_filter
      & inpatient_beds_7_day_avg_filter]
print(df2_filter.head(20))

                                              hospital_pk collection_week  \
28847   3b081d5ef1c552538e4af4aa593a857bb922a4f364a412...         1/28/22   
28884                                              190307         1/28/22   
32626                                              191325         1/28/22   
38387                                              330224          1/7/22   
310367                                             330403          1/1/21   
338062                                             360080        11/27/20   
353610                                         45C0001441         11/6/20   
357121                                             412001        10/16/20   

       state         ccn                                hospital_name  \
28847     LA         NaN                    Surgery Center of Zachary   
28884     LA      190307              SOUTH CAMERON MEMORIAL HOSPITAL   
32626     LA      191325             LADY OF THE SEA GENERAL HOSPITAL   
38387     NY  